In [1]:
import numpy as np

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold

# Step 1: Load the data into a DataFrame
df = pd.read_excel('/projects/cc/se_users/spreng_klqh882/rct/Randomised Coffee Trials - Q1 2025(1-87).xlsx')

# Step 2: Select the columns we need for stratification
position_col = "Which of the following best describes your position?"
job_family_col = "Which of the following best describes your Job Family Group?"

# Step 3: Define mappings for merging classes
position_mapping = {
    "Master's student": "Student/Early Career",
    "PhD student": "Student/Early Career",
    "Graduate": "Student/Early Career",
    "Intern": "Student/Early Career",
    "Postdoc": "Student/Early Career",
    "Post doc": "Student/Early Career",
    "Placement student": "Student/Early Career",
    "Entry-level position (<3 years of experience)": "Entry-level",
    "Mid-level position (3-10 years experience)": "Mid-level",
    "Senior-level position (>10 years experience)": "Senior-level",
    "Senior position (>10 years experience)": "Senior-level",
}

job_family_mapping = {
    "Clinical trials": "Operations & Support",
    "Data and AI": "Technology",
    "Information Technology": "Technology",
    "Pharmaceutical Science": "Pharmaceutical Science",
    "Chemistry": "Science",
    "Science generic": "Science",
    "DMPK": "Science",
    "Intellectual Property": "Business",
    "Quality": "Operations & Support",
    "Regulatory": "Operations & Support",
    "Human Resources": "Operations & Support",
    "Management": "Operations & Support",
    "Bioscience": "Bioscience",
}



In [3]:

# Step 4: Apply the mappings to create new columns with merged classes
df['position_merged'] = df[position_col].map(position_mapping).fillna("Other")
df['job_family_merged'] = df[job_family_col].map(job_family_mapping).fillna("Other")

# Print initial counts
print("\nInitial counts for merged job family column:")
print(df['job_family_merged'].value_counts())

# Step 5: Move the one person from "Other" to "Senior-level" in position grouping
other_position_index = df[df['position_merged'] == "Other"].index
if len(other_position_index) == 1:
    df.loc[other_position_index, 'position_merged'] = "Senior-level"
elif len(other_position_index) > 1:
    print("Warning: More than one person in 'Other' position category")

# Step 6: Randomly move people from mid-level to senior-level if needed
np.random.seed(42)  # for reproducibility
senior_count = (df['position_merged'] == 'Senior-level').sum()
mid_level_indices = df[df['position_merged'] == 'Mid-level'].index
num_to_move = 18 - senior_count  # Adjusted to 18 based on our last discussion

if num_to_move > 0:
    indices_to_move = np.random.choice(mid_level_indices, size=num_to_move, replace=False)
    df.loc[indices_to_move, 'position_merged'] = 'Senior-level'

# Step 7: Move 5 from "Other" to "Science", 5 to "Technology", all from "Business" to "Science", and one more from "Pharmaceutical Science" to "Science"
other_indices = df[df['job_family_merged'] == 'Other'].index
business_indices = df[df['job_family_merged'] == 'Business'].index
pharma_indices = df[df['job_family_merged'] == 'Pharmaceutical Science'].index

print(f"\nNumber of people in 'Other' group: {len(other_indices)}")

# Move 5 from "Other" to "Science" and 5 to "Technology"
if len(other_indices) >= 10:
    other_to_science = np.random.choice(other_indices, size=5, replace=False)
    df.loc[other_to_science, 'job_family_merged'] = 'Science'
    
    other_indices = df[df['job_family_merged'] == 'Other'].index  # Update other_indices
    other_to_technology = np.random.choice(other_indices, size=5, replace=False)
    df.loc[other_to_technology, 'job_family_merged'] = 'Technology'
else:
    print(f"Warning: Only {len(other_indices)} people in 'Other' group. Moving all to Science and Technology.")
    half = len(other_indices) // 2
    df.loc[other_indices[:half], 'job_family_merged'] = 'Science'
    df.loc[other_indices[half:], 'job_family_merged'] = 'Technology'

# Move all from "Business" to "Science"
df.loc[business_indices, 'job_family_merged'] = 'Science'

# Move one more from "Pharmaceutical Science" to "Science"
if len(pharma_indices) >= 1:
    pharma_to_move = np.random.choice(pharma_indices, size=1, replace=False)
    df.loc[pharma_to_move, 'job_family_merged'] = 'Science'
else:
    print("Warning: Not enough people in 'Pharmaceutical Science' group to move one more to 'Science'")

# Print updated counts
print("\nUpdated counts for merged job family column:")
print(df['job_family_merged'].value_counts())


Initial counts for merged job family column:
job_family_merged
Operations & Support      19
Pharmaceutical Science    18
Bioscience                17
Technology                12
Science                   11
Other                      9
Business                   1
Name: count, dtype: int64

Number of people in 'Other' group: 9

Updated counts for merged job family column:
job_family_merged
Operations & Support      19
Technology                17
Pharmaceutical Science    17
Science                   17
Bioscience                17
Name: count, dtype: int64


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Load and preprocess data (Steps 1-7 remain the same)
# ... [Keep all the code up to Step 7 as it is] ...

# Function to calculate diversity score
def calculate_diversity_score(df, groups):
    score = 0
    for group in groups:
        position_diversity = len(set(df.loc[group, 'position_merged']))
        job_family_diversity = len(set(df.loc[group, 'job_family_merged']))
        score += position_diversity + job_family_diversity
    return score

# Run stratification 100 times
best_score = -1
best_groups = None

for i in range(100):
    skf = StratifiedKFold(n_splits=17, shuffle=True, random_state=i)
    groups = list(skf.split(df, df['position_merged']))
    
    # Convert split indices to actual dataframe indices
    groups = [df.index[test].tolist() for _, test in groups]
    
    # Ensure each group has exactly 5 members
    for j, group in enumerate(groups):
        if len(group) > 5:
            groups[j] = group[:5]
        elif len(group) < 5:
            remaining = [idx for idx in df.index if all(idx not in g for g in groups)]
            groups[j].extend(remaining[:5-len(group)])
    
    score = calculate_diversity_score(df, groups)
    if score > best_score:
        best_score = score
        best_groups = groups

# Assign the best groups to the dataframe
df['group'] = -1
for i, group in enumerate(best_groups):
    df.loc[group, 'group'] = i

# Print results
print("\nBest partition found:")
for i, group in enumerate(best_groups):
    print(f"\nGroup {i+1}:")
    print(df.loc[group, ['position_merged', 'job_family_merged']])
    print(f"Group size: {len(group)}")

print("\nOverall group statistics:")
print(df.groupby('group').agg({
    'position_merged': 'nunique',
    'job_family_merged': 'nunique',
    'group': 'count'
}).rename(columns={
    'position_merged': 'Unique Positions',
    'job_family_merged': 'Unique Job Families',
    'group': 'Group Size'
}))

# Check for any unassigned individuals
unassigned = df[df['group'] == -1]
if not unassigned.empty:
    print("\nWarning: Some individuals were not assigned to groups:")
    print(unassigned[['position_merged', 'job_family_merged']])
else:
    print("\nAll individuals have been assigned to groups.")

# Optional: Save the results to a new CSV file
df.to_csv('event_data_with_groups_merged.csv', index=False)

# Print the distribution of merged classes in the best partition
print("\nDistribution of merged positions in best partition:")
print(df['position_merged'].value_counts())
print("\nDistribution of merged job families in best partition:")
print(df['job_family_merged'].value_counts())


Best partition found:

Group 1:
         position_merged       job_family_merged
16           Entry-level                 Science
21             Mid-level                 Science
46  Student/Early Career  Pharmaceutical Science
50          Senior-level              Technology
62  Student/Early Career              Technology
Group size: 5

Group 2:
         position_merged       job_family_merged
5            Entry-level                 Science
17             Mid-level    Operations & Support
22  Student/Early Career  Pharmaceutical Science
26          Senior-level  Pharmaceutical Science
33  Student/Early Career              Bioscience
Group size: 5

Group 3:
         position_merged       job_family_merged
40          Senior-level  Pharmaceutical Science
52           Entry-level              Technology
57  Student/Early Career                 Science
64  Student/Early Career              Bioscience
70             Mid-level              Technology
Group size: 5

Group 4:
         posi

In [5]:
"""
import pandas as pd
import numpy as np

def assign_remaining_to_maximize_diversity(df):
    def count_unique_job_families(group):
        return group['job_family_merged'].nunique()

    unassigned = df[df['group'] == -1]
    
    # Count the current size of each group
    group_sizes = df['group'].value_counts()
    
    # Determine the target group size
    target_size = (len(df) + len(unassigned)) // 17  # 17 is the number of groups
    
    for idx, row in unassigned.iterrows():
        max_unique_jobs = -1
        best_group = -1
        
        # Prioritize groups that are below the target size
        available_groups = [g for g in range(17) if group_sizes.get(g, 0) < target_size]
        
        # If all groups are at or above target size, consider all groups
        if not available_groups:
            available_groups = range(17)
        
        for group in available_groups:
            temp_group = df[df['group'] == group]['job_family_merged'].tolist() + [row['job_family_merged']]
            unique_jobs = len(set(temp_group))
            
            # Tiebreaker: prefer smaller groups if unique job count is the same
            if unique_jobs > max_unique_jobs or (unique_jobs == max_unique_jobs and group_sizes.get(group, 0) < group_sizes.get(best_group, float('inf'))):
                max_unique_jobs = unique_jobs
                best_group = group

        df.loc[idx, 'group'] = best_group
        group_sizes[best_group] = group_sizes.get(best_group, 0) + 1

    return df

# Apply the function to assign remaining individuals
df = assign_remaining_to_maximize_diversity(df)

# ... (rest of the code remains the same)

# Print results
print("\nUpdated group assignments:")
for group in range(17):
    group_df = df[df['group'] == group]
    print(f"\nGroup {group}:")
    print(group_df[['position_merged', 'job_family_merged']])
    print(f"Group size: {len(group_df)}")
    print(f"Unique job families: {group_df['job_family_merged'].nunique()}")

print("\nOverall group statistics:")
group_stats = df.groupby('group').agg({
    'position_merged': 'nunique',
    'job_family_merged': 'nunique',
    'group': 'count'
}).rename(columns={
    'position_merged': 'Unique Positions',
    'job_family_merged': 'Unique Job Families',
    'group': 'Group Size'
})
print(group_stats)

# Check if all individuals are now assigned
if (df['group'] == -1).sum() == 0:
    print("\nAll individuals have been assigned to groups.")
else:
    print("\nWarning: Some individuals are still not assigned to groups.")

# Print the final distribution of merged classes
print("\nFinal distribution of merged positions:")
print(df['position_merged'].value_counts())
print("\nFinal distribution of merged job families:")
print(df['job_family_merged'].value_counts())

# Calculate and print diversity metrics
total_unique_job_families = group_stats['Unique Job Families'].sum()
average_unique_job_families = group_stats['Unique Job Families'].mean()
min_unique_job_families = group_stats['Unique Job Families'].min()
max_unique_job_families = group_stats['Unique Job Families'].max()

print("\nDiversity Metrics:")
print(f"Total unique job families across all groups: {total_unique_job_families}")
print(f"Average unique job families per group: {average_unique_job_families:.2f}")
print(f"Minimum unique job families in a group: {min_unique_job_families}")
print(f"Maximum unique job families in a group: {max_unique_job_families}")
"""

'\nimport pandas as pd\nimport numpy as np\n\ndef assign_remaining_to_maximize_diversity(df):\n    def count_unique_job_families(group):\n        return group[\'job_family_merged\'].nunique()\n\n    unassigned = df[df[\'group\'] == -1]\n    \n    # Count the current size of each group\n    group_sizes = df[\'group\'].value_counts()\n    \n    # Determine the target group size\n    target_size = (len(df) + len(unassigned)) // 17  # 17 is the number of groups\n    \n    for idx, row in unassigned.iterrows():\n        max_unique_jobs = -1\n        best_group = -1\n        \n        # Prioritize groups that are below the target size\n        available_groups = [g for g in range(17) if group_sizes.get(g, 0) < target_size]\n        \n        # If all groups are at or above target size, consider all groups\n        if not available_groups:\n            available_groups = range(17)\n        \n        for group in available_groups:\n            temp_group = df[df[\'group\'] == group][\'job_fami

In [6]:
import pandas as pd
import numpy as np

def reassign_to_new_group(df):
    # Identify unassigned individuals
    unassigned = df[df['group'] == -1]
    
    # Create a new group (group 17)
    new_group = 17
    
    # Assign unassigned individuals to the new group
    df.loc[df['group'] == -1, 'group'] = new_group
    
    # Count how many more we need to make a group of 4
    needed = max(0, 4 - len(unassigned))
    
    if needed > 0:
        # Find groups with 5 members
        groups_of_five = df['group'].value_counts()[df['group'].value_counts() == 5].index.tolist()
        
        for _ in range(needed):
            if not groups_of_five:
                break
            
            max_diversity_gain = -1
            best_member_to_move = None
            best_source_group = None
            
            for group in groups_of_five:
                group_df = df[df['group'] == group]
                new_group_df = df[df['group'] == new_group]
                
                for _, member in group_df.iterrows():
                    # Calculate diversity before move
                    diversity_before = (len(set(group_df['position_merged'])) + 
                                        len(set(new_group_df['position_merged'])))
                    
                    # Calculate diversity after potential move
                    diversity_after = (len(set(group_df['position_merged'].drop(member.name))) + 
                                       len(set(new_group_df['position_merged'].tolist() + [member['position_merged']])))
                    
                    diversity_gain = diversity_after - diversity_before
                    
                    if diversity_gain > max_diversity_gain:
                        max_diversity_gain = diversity_gain
                        best_member_to_move = member
                        best_source_group = group
            
            if best_member_to_move is not None:
                # Move the best member to the new group
                df.loc[best_member_to_move.name, 'group'] = new_group
                
                # Remove the source group from groups_of_five if it now has 4 members
                if (df['group'] == best_source_group).sum() == 4:
                    groups_of_five.remove(best_source_group)
            else:
                # If we can't improve diversity, just move a random member
                group = groups_of_five[0]
                member_to_move = df[df['group'] == group].sample(n=1)
                df.loc[member_to_move.index, 'group'] = new_group
                if (df['group'] == group).sum() == 4:
                    groups_of_five.remove(group)
    
    return df

# Apply the function to reassign individuals
df = reassign_to_new_group(df)

# Print results
print("\nUpdated group assignments:")
for group in range(18):  # Now we have 18 groups
    group_df = df[df['group'] == group]
    print(f"\nGroup {group}:")
    print(group_df[['position_merged', 'job_family_merged']])
    print(f"Group size: {len(group_df)}")
    print(f"Unique positions: {group_df['position_merged'].nunique()}")
    print(f"Unique job families: {group_df['job_family_merged'].nunique()}")

print("\nOverall group statistics:")
group_stats = df.groupby('group').agg({
    'position_merged': 'nunique',
    'job_family_merged': 'nunique',
    'group': 'count'
}).rename(columns={
    'position_merged': 'Unique Positions',
    'job_family_merged': 'Unique Job Families',
    'group': 'Group Size'
})
print(group_stats)

# Check if all individuals are now assigned
if (df['group'] == -1).sum() == 0:
    print("\nAll individuals have been assigned to groups.")
else:
    print("\nWarning: Some individuals are still not assigned to groups.")

# Print the final distribution of merged classes
print("\nFinal distribution of merged positions:")
print(df['position_merged'].value_counts())
print("\nFinal distribution of merged job families:")
print(df['job_family_merged'].value_counts())

# Calculate and print diversity metrics
total_unique_positions = group_stats['Unique Positions'].sum()
average_unique_positions = group_stats['Unique Positions'].mean()
min_unique_positions = group_stats['Unique Positions'].min()
max_unique_positions = group_stats['Unique Positions'].max()

print("\nPosition Diversity Metrics:")
print(f"Total unique positions across all groups: {total_unique_positions}")
print(f"Average unique positions per group: {average_unique_positions:.2f}")
print(f"Minimum unique positions in a group: {min_unique_positions}")
print(f"Maximum unique positions in a group: {max_unique_positions}")


Updated group assignments:

Group 0:
         position_merged       job_family_merged
16           Entry-level                 Science
21             Mid-level                 Science
46  Student/Early Career  Pharmaceutical Science
50          Senior-level              Technology
62  Student/Early Career              Technology
Group size: 5
Unique positions: 4
Unique job families: 3

Group 1:
         position_merged       job_family_merged
5            Entry-level                 Science
17             Mid-level    Operations & Support
22  Student/Early Career  Pharmaceutical Science
26          Senior-level  Pharmaceutical Science
33  Student/Early Career              Bioscience
Group size: 5
Unique positions: 4
Unique job families: 4

Group 2:
         position_merged       job_family_merged
40          Senior-level  Pharmaceutical Science
52           Entry-level              Technology
57  Student/Early Career                 Science
64  Student/Early Career              Biosci

In [7]:
import pandas as pd

def assign_groups_and_save(df, output_file='group_assignments.csv'):
    # Ensure the 'group' column exists in the DataFrame
    if 'group' not in df.columns:
        raise ValueError("The DataFrame does not have a 'group' column. Please assign groups before calling this function.")

    # Save the DataFrame to a file
    if output_file.endswith('.csv'):
        df.to_csv(output_file, index=False)
    elif output_file.endswith('.xlsx'):
        df.to_excel(output_file, index=False)
    else:
        raise ValueError("Output file must be either .csv or .xlsx")

    print(f"Results have been saved to {output_file}")

    # Return the DataFrame
    return df

# Usage:
# Assuming 'df' is your DataFrame with the final group assignments in a 'group' column
df = assign_groups_and_save(df, 'group_assignments.csv')  # For CSV
# df = assign_groups_and_save(df, 'group_assignments.xlsx')  # For Excel

Results have been saved to group_assignments.csv


In [8]:
import pandas as pd

def save_group_details_to_file(original_excel_path, group_assignments_csv_path, output_txt_path):
    # Read the original Excel file
    original_df = pd.read_excel(original_excel_path)

    # Read the group assignments CSV file
    group_assignments = pd.read_csv(group_assignments_csv_path)

    # Merge the original data with group assignments using 'Name2' column
    df = original_df.merge(group_assignments[['Name2', 'group']], on='Name2', how='left')

    # Sort the DataFrame by group to ensure groups are processed in order
    df = df.sort_values('group')

    # Get the number of unique groups
    num_groups = df['group'].nunique()

    # Open the output file
    with open(output_txt_path, 'w') as f:
        f.write("Group Details\n")
        f.write("=" * 50 + "\n\n")

        # Iterate through each group
        for group in range(num_groups):
            f.write(f"Group {group + 1}:\n")
            f.write("-" * 50 + "\n")

            # Filter the DataFrame for the current group
            group_df = df[df['group'] == group]

            # Iterate through each person in the group
            for _, row in group_df.iterrows():
                name = row['Name2']
                position = row["Which of the following best describes your position?"]
                job_family = row["Which of the following best describes your Job Family Group?"]

                f.write(f"Name: {name}\n")
                f.write(f"Position: {position}\n")
                f.write(f"Job Family: {job_family}\n")
                f.write("\n")  # Empty line for readability between people

            f.write(f"Total members in Group {group + 1}: {len(group_df)}\n")
            f.write("-" * 50 + "\n\n")

    print(f"Group details have been saved to {output_txt_path}")

    # Print summary of unmatched individuals
    unmatched = df[df['group'].isna()]
    if not unmatched.empty:
        print(f"\nWarning: {len(unmatched)} individuals could not be matched to a group:")
        print(unmatched[['Name2', "Which of the following best describes your position?", "Which of the following best describes your Job Family Group?"]])

# Usage:
save_group_details_to_file(
    '/projects/cc/se_users/spreng_klqh882/rct/Randomised Coffee Trials - Q1 2025(1-87).xlsx',
    '/projects/cc/se_users/spreng_klqh882/rct/group_assignments.csv',
    'group_details_output.txt'
)

Group details have been saved to group_details_output.txt


In [9]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils import get_column_letter
import colorsys
import random

def generate_divergent_colors(n):
    def hsv_to_hex(h, s, v):
        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        return '{:02x}{:02x}{:02x}'.format(int(r * 255), int(g * 255), int(b * 255))

    hue_step = 0.618033988749895  # Golden ratio conjugate
    hue_start = random.random()
  
    colors = []
    for i in range(n):
        hue = (hue_start + i * hue_step) % 1
        saturation = 0.5 + (random.random() * 0.3)  # Vary saturation
        value = 0.95 - (random.random() * 0.3)  # Vary value, but keep it light for readability
        colors.append(hsv_to_hex(hue, saturation, value))
  
    random.shuffle(colors)  # Shuffle to ensure adjacent colors are different
    return colors

def lighten_color(color, factor=0.7):
    # Convert hex to RGB
    r = int(color[:2], 16)
    g = int(color[2:4], 16)
    b = int(color[4:], 16)
    
    # Lighten the color
    r = int(r + (255 - r) * factor)
    g = int(g + (255 - g) * factor)
    b = int(b + (255 - b) * factor)
    
    # Convert back to hex
    return '{:02x}{:02x}{:02x}'.format(r, g, b)

def save_group_details_to_excel(input_csv_path, output_excel_path):
    # Read the CSV file
    df = pd.read_csv(input_csv_path)

    # Select and rename the columns we want
    df_output = df[['group', 'Name2', 'Email (For AZ employees, use your.name@astrazeneca.com, but others are welcome too)',  
                'Which of the following best describes your position?',  
                'Which of the following best describes your Job Family Group?']]
    df_output.columns = ['Group', 'Name', 'Email', 'Position', 'Job Family']

    # Sort by group
    df_output = df_output.sort_values('Group')

    # Create a new workbook and select the active sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Group Details"

    # Write the header
    headers = ['Group', 'Name', 'Email', 'Position', 'Job Family']
    for col, header in enumerate(headers, start=1):
        cell = ws.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.alignment = Alignment(horizontal='center')

    # Generate colors for each group
    unique_groups = sorted(df_output['Group'].unique())
    colors = generate_divergent_colors(len(unique_groups))
    color_map = dict(zip(unique_groups, colors))

    # Define border styles
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    thick_bottom_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thick'))

    # Write the data and apply conditional formatting
    current_group = None
    for row, (_, data) in enumerate(df_output.iterrows(), start=2):
        group_color = color_map[data['Group']]
        light_color = lighten_color(group_color, 0.7)  # 70% lighter
        
        for col, value in enumerate(data, start=1):
            cell = ws.cell(row=row, column=col, value=value)
            cell.alignment = Alignment(horizontal='left')
            
            if col == 1:  # Group column
                cell.fill = PatternFill(start_color=group_color, end_color=group_color, fill_type="solid")
            else:  # Other columns
                cell.fill = PatternFill(start_color=light_color, end_color=light_color, fill_type="solid")

            # Apply borders
            if current_group is not None and data['Group'] != current_group:
                # This is the first row of a new group, apply thick bottom border to the previous row
                for prev_col in range(1, 6):
                    ws.cell(row=row-1, column=prev_col).border = thick_bottom_border
            else:
                cell.border = thin_border

        current_group = data['Group']

    # Apply thick bottom border to the last row
    for col in range(1, 6):
        ws.cell(row=ws.max_row, column=col).border = thick_bottom_border

    # Adjust column widths
    ws.column_dimensions['A'].width = 10
    ws.column_dimensions['B'].width = 30
    ws.column_dimensions['C'].width = 40  # Email column
    ws.column_dimensions['D'].width = 40
    ws.column_dimensions['E'].width = 40

    # Save the workbook
    wb.save(output_excel_path)
    print(f"Group details have been saved to {output_excel_path}")


# Usage
input_csv_path = '/projects/cc/se_users/spreng_klqh882/rct/group_assignments.csv'
output_excel_path = 'RCT_AutoGroupAssignments.xlsx'
save_group_details_to_excel(input_csv_path, output_excel_path)

Group details have been saved to RCT_AutoGroupAssignments.xlsx
